# Fine align
---

#### Overview
Interactive 3D alignment of serial sections.

In [1]:
import pathlib
import requests

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import renderapi
import os

from scripted_render_pipeline import basic_auth
from scripted_render_pipeline.importer import uploader
from interactive_render import plotting
from interactive_render.utils import clear_image_cache
from interactive_render.utils import get_stitching_pointmatches, get_alignment_pointmatches

In [2]:
clear_image_cache()

<Response [200]>

#### `render-ws` environment variables

In [3]:
# create an authorized session
auth = basic_auth.load_auth()
sesh = requests.Session()
sesh.auth = auth

# render-ws environment variables
params_render = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "session": sesh
}

params_align = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "memGB": '40G',
    "session": sesh
}

params_uploader = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "auth": auth
}

# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/")

## 5) Fine alignment
---

In [4]:
stacks = {
    'rough_align': 'postcorrection_aligned_rough',
    'in': 'postcorrection',
    'out': 'postcorrection_rigid'
}

### Get point matches

Use `render-ws` `PointMatchClient` script to find point matches between tilepairs in `z`, based on rough alignment
#### Collect tile pairs

In [5]:
# Get z values
z_values = renderapi.stack.get_z_values_for_stack(stack=stacks['rough_align'],
                                                  **params_render)

# Get tile pairs
tile_pairs = renderapi.client.tilePairClient(stack=stacks['rough_align'],
                                             minz=min(z_values),
                                             maxz=max(z_values),
                                             zNeighborDistance=1, # half-height of search cylinder
                                             xyNeighborFactor=0.1,
                                             excludeSameLayerNeighbors=True,
                                             subprocess_mode="check_output",  # suppresses output
                                             **params_render)
# Reformat tilepairs for PointMatchClient
tile_pairs_reformat = [[tp['p']['id'], tp['q']['id']] for tp in tile_pairs['neighborPairs']]

# Show tile pairs
out = f"Number of tile pairs... {len(tile_pairs_reformat)}"
print(out, "\n" + "-"*len(out))
tile_pairs_reformat[0:5]

Number of tile pairs... 53 
--------------------------


[['t00_z0_y4_x4', 't06_z1_y3_x3'],
 ['t01_z0_y4_x3', 't07_z1_y3_x2'],
 ['t02_z0_y4_x2', 't08_z1_y3_x1'],
 ['t03_z0_y4_x1', 't09_z1_y3_x0'],
 ['t05_z0_y3_x4', 't11_z1_y2_x3']]

In [10]:
# Name for pointmatch collection
match_collection = f"{params_render['project']}_{stacks['in']}_align_matches"
match_collection

'20230914_RP_exocrine_partial_test_total_matches'

#### Set `SIFT` & `RANSAC` parameters for fine alignment

In [30]:
from renderapi.client import (
    SiftPointMatchOptions,
    MatchDerivationParameters,
    FeatureExtractionParameters
)

In [31]:
# `RANSAC` parameters
params_RANSAC = MatchDerivationParameters(
    matchIterations=None,
    matchMaxEpsilon=25,        # maximal alignment error
    matchMaxNumInliers=None,
    matchMaxTrust=None,
    matchMinInlierRatio=0.05,  # minimal inlier ratio
    matchMinNumInliers=7,      # minimal number of inliers
    matchModelType='AFFINE',   # expected transformation
    matchRod=0.92              # closest/next closest ratio
)

# `SIFT` parameters
params_SIFT = FeatureExtractionParameters(
    SIFTfdSize=8,              # feature descriptor size
    SIFTmaxScale=0.2,         # (width/height *) maximum image size
    SIFTminScale=0.05,         # (width/height *) minimum image size
    SIFTsteps=5               # steps per scale octave
)

# Combined `SIFT` & `RANSAC` parameters
params_SIFT = SiftPointMatchOptions(
    fillWithNoise=True,
    **{**params_RANSAC.__dict__,
       **params_SIFT.__dict__}
)

# Extra parameters
params_SIFT.numberOfThreads = 20 # multithreading
params_SIFT.__dict__

{'SIFTfdSize': 8,
 'SIFTmaxScale': 0.2,
 'SIFTminScale': 0.05,
 'SIFTsteps': 5,
 'matchIterations': None,
 'matchMaxEpsilon': 25,
 'matchMaxNumInliers': None,
 'matchMaxTrust': None,
 'matchMinInlierRatio': 0.05,
 'matchMinNumInliers': 7,
 'matchModelType': 'AFFINE',
 'matchRod': 0.92,
 'renderScale': None,
 'fillWithNoise': True,
 'numberOfThreads': 20}

#### Get point matches

In [32]:
batch_size = params_SIFT.numberOfThreads # batch size equal to # of threads used
batches = [i for i in range(0, len(tile_pairs_reformat)+1, batch_size)]

In [32]:
for i in tqdm((batches),
              total=len(batches)):
    tile_pair_batch = tile_pairs_reformat[i:(i+batch_size)]

    # Run SIFT + RANSAC via render-ws PointMatchClient
    renderapi.client.pointMatchClient(
        stack=stacks['in'],
        collection=match_collection,
        tile_pairs=tile_pair_batch,
        sift_options=params_SIFT,
        excludeAllTransforms=True,
        subprocess_mode='check_output',  # suppresses output
        **params_align
    )

  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
renderapi.pointmatch.delete_collection(match_collection,
                                       **params_render)

#### Inspect matches

In [8]:
from interactive_render import plotting

In [9]:
plotting.plot_aligned_stack_with_alignment_matches(
    stacks['in'],
    match_collection,
    width=1000,
    **params_render
)

interactive(children=(IntSlider(value=0, description='z', max=1), Output()), _dom_classes=('widget-interact',)…

### Combine stitching and aligment match collections
Facilitate alignment using both match collections

In [36]:
# Name for stitch and align pointmatch collection
match_collection_total = f"{params_render['project']}_{stacks['in']}_total_matches"
match_collection_total

'20230914_RP_exocrine_partial_test_postcorrection_total_matches'

In [42]:
# Get all point matches
# get intrasection point matches
stitch_matches = get_stitching_pointmatches(
    "postcorrection",
    f"{params_render['project']}_{stacks['in']}_stitch_matches",
    **params_render
    )

# get intersection point matches
align_matches = get_alignment_pointmatches(
    "postcorrection",
    f"{params_render['project']}_{stacks['in']}_align_matches",
    **params_render
    )

# Combine the lot
all_matches = []
z_values = renderapi.stack.get_z_values_for_stack(stack="postcorrection",
                                                  **params_render)
for z in z_values:
    all_matches += stitch_matches[z]

for z in z_values:
    all_matches += align_matches[z]
    
# Import master collection of matches
renderapi.pointmatch.import_matches(
    match_collection_total,
    all_matches,
    **params_render
)

<Response [201]>

### Align stack
---

### Create alignment files

In [43]:
from pathlib import Path
import os
import subprocess
import json
from pprint import pprint

In [44]:
z_values = renderapi.stack.get_z_values_for_stack(stacks['in'],
                                                  **params_render)
# Load align.json template
template_align_json = Path('/home/akievits/interactive-render-workflow/templates/align.json')
with template_align_json.open('r') as json_data:
    params_align_fine = json.load(json_data)

# Edit BigFeta solver schema
params_align_fine['first_section'] = min(z_values)
params_align_fine['last_section'] = max(z_values)
params_align_fine['solve_type'] = '3D'
params_align_fine['transformation'] = 'SimilarityModel'
params_align_fine['log_level'] = 'INFO'

# Edit input stack data
params_align_fine['input_stack']['host'] = params_render['host']
params_align_fine['input_stack']['owner'] = params_render['owner']
params_align_fine['input_stack']['project'] = params_render['project']
params_align_fine['input_stack']['name'] = stacks['in']

# Edit point match stack data
params_align_fine['pointmatch']['host'] = params_render['host']
params_align_fine['pointmatch']['owner'] = params_render['owner']
params_align_fine['pointmatch']['name'] = match_collection_total

# Edit output stack data
params_align_fine['output_stack']['host'] = params_render['host']
params_align_fine['output_stack']['owner'] = params_render['owner']
params_align_fine['output_stack']['project'] = params_render['project']
params_align_fine['output_stack']['name'] = stacks['out']

# Edit alignment parameters
params_align_fine['matrix_assembly']['depth'] = 2
params_align_fine['matrix_assembly']['montage_pt_weight'] = 0.5
params_align_fine['matrix_assembly']['cross_pt_weight'] = 1
params_align_fine['matrix_assembly']['npts_min'] = 5
params_align_fine['matrix_assembly']['npts_max'] = 100

# Edit regularization parameters
params_align_fine['regularization']['default_lambda'] = 1e3    # default: 0.005
params_align_fine['regularization']['translation_factor'] = 0.00001  # default: 0.005
params_align_fine['regularization']['thinplate_factor'] = 1e-5      # default: 1e-5

# Export montage settings to
align_json = dir_project / '_jsons_align_fine' / stacks['in'] / 'align_fine.json'
align_json.parent.mkdir(parents=True, exist_ok=True)
with align_json.open('w') as json_data:
    json.dump(params_align_fine, json_data, indent=2)

# Check alignment parameters
print(align_json)
print('-'*len(align_json.as_posix()))
pprint(params_align_fine)

/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/_jsons_align_fine/postcorrection/align_fine.json
----------------------------------------------------------------------------------------------------------------------
{'close_stack': 'True',
 'first_section': 0.0,
 'hdf5_options': {'chunks_per_file': -1, 'output_dir': ''},
 'input_stack': {'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
                 'collection_type': 'stack',
                 'db_interface': 'render',
                 'host': 'http://localhost',
                 'mongo_host': 'sonic.tnw.tudelft.nl',
                 'mongo_port': 27017,
                 'name': 'postcorrection',
                 'owner': 'akievits',
                 'port': 8081,
                 'project': '20230914_RP_exocrine_partial_test'},
 'last_section': 2.0,
 'log_level': 'INFO',
 'matrix_assembly': {'choose_random': 'False',
                     'cross_pt_weight': 1,
                   

### Run `BigFeta`

In [45]:
# Path to `BigFeta`
cwd = Path.cwd().as_posix()
BigFeta_dir = Path('/home/catmaid/BigFeta/')

# Call `BigFeta.BigFeta` process -- have to switch to BigFeta directory
os.chdir(BigFeta_dir.as_posix())
subprocess.run(['python', '-m', 'bigfeta.bigfeta', '--input_json', align_json.as_posix()])
os.chdir(cwd)

/home/akievits/interactive-render-workflow/.venv/lib/python3.9/site-packages/argschema/utils.py:346: FutureWarning: '--transform_apply' is using old-style command-line syntax with each element as a separate argument. This will not be supported in argschema after 2.0. See http://argschema.readthedocs.io/en/master/user/intro.html#command-line-specification for details.
  warnings.warn(warn_msg, FutureWarning)
INFO:bigfeta.utils:
 loaded 75 tile specs from 3 zvalues in 0.1 sec using interface: render
INFO:__main__: A created in 0.4 seconds
INFO:__main__:
 solved in 0.0 sec
 precision [norm(Kx-Lm)/norm(Lm)] = 3.4e-09
 error     [norm(Ax-b)] = 1437.089
 [mean(Ax) +/- std(Ax)] : 0.0 +/- 7.2
 [mean(error mag) +/- std(error mag)] : 4.9 +/- 5.3
/home/catmaid/BigFeta/bigfeta/transform/similarity_model.py:68: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this opera

In [46]:
clear_image_cache()

<Response [200]>

### Inspect fine alignment

In [7]:
# plot stack
plotting.plot_stacks(
    stacks=[stacks['out']],
    width=1000,
    vmin=0,
    vmax=65535,
    **params_render
)

  0%|          | 0/1 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=2), IntSlider(value=26558, description='vmin', m…

### Inspect fine alignment
Plot tiles to qualitatively assess alignment quality

In [9]:
plotting.plot_aligned_tiles(stack=stacks['out'],
                            width=2000,
                            **params_render)

interactive(children=(IntSlider(value=0, description='z', max=1), FloatSlider(value=0.5, description='alpha1',…

### Update stack metadata

In [48]:
from interactive_render.utils import update_stack_resolution

In [49]:
update_stack_resolution(stack=stacks['out'],
                        stackResolutionX=4.0,
                        stackResolutionY=4.0,
                        stackResolutionZ=80.0,
                        **params_render)

In [50]:
renderapi.stack.set_stack_state(stacks['out'],
                                state='COMPLETE',
                                **params_render)

<Response [201]>

### Update stack bounds
Set minimum X and Y bounds to 0 because WebKnossos requires it

In [16]:
bounds = renderapi.stack.get_stack_bounds(stacks['out'],
                                          **params_render)
tX = -1*bounds['minX']
tY = -1*bounds['minY']

ts_new = []
z_values = renderapi.stack.get_z_values_for_stack(stacks['out'],
                                                  **params_render)
for z in tqdm(z_values, total=len(z_values)):
    # Get tilespec and transformations from the aligned stack
    TileSpec = renderapi.tilespec.get_tile_specs_from_z(stacks['out'],
                                                        z=z,
                                                        **params_render)
    translation = renderapi.transform.leaf.AffineModel(B0=tX, B1=tY)
    # Translate stack
    for ts in TileSpec:
        ts.tforms += [translation]
        ts_new.append(ts)
        
# Create new stack
renderapi.stack.create_stack(f"{stacks['out']}_translated",
                             **params_render)
# Import tilespecs to stack
renderapi.client.import_tilespecs(f"{stacks['out']}_translated",
                                  tilespecs=ts_new,
                                  **params_render)
# Close stack
renderapi.stack.set_stack_state(f"{stacks['out']}_translated",
                                state='COMPLETE',
                                **params_render)

  0%|          | 0/3 [00:00<?, ?it/s]

org.janelia.render.client.ImportJsonClient

  Running: /home/catmaid/render/deploy/jdk1.8.0_131/bin/java -cp /home/catmaid/render/render-ws-java-client/target/render-ws-java-client-2.0.1-SNAPSHOT-standalone.jar -Xms1G -Xmx1G -Djava.awt.headless=true -XX:+UseSerialGC org.janelia.render.client.ImportJsonClient --baseDataUrl http://localhost:8081/render-ws/v1 --owner akievits --project 20230914_RP_exocrine_partial_test --stack postcorrection_rigid_translated /tmp/tmpetapfiew.json


11:21:29.395 [main] INFO  [org.janelia.render.client.ClientRunner] run: entry
11:21:29.636 [main] INFO  [org.janelia.render.client.ImportJsonClient] runClient: entry, parameters={
  "renderWeb" : {
    "baseDataUrl" : "http://localhost:8081/render-ws/v1",
    "owner" : "akievits",
    "project" : "20230914_RP_exocrine_partial_test"
  },
  "tileSpecValidator" : { },
  "stack" : "postcorrection_rigid_translated",
  "tileFiles" : [
    "/tmp/tmpetapfiew.json"
  ]
}
11:21:29.997 [main] INFO  [org.janelia.render.cli

<Response [201]>